# NBA(미국 농구 리그) 웹사이트에서 다음 경기일정 찾기

## BeautifulSoup으로 경기날짜, 장소, 팀 가져오기

In [48]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [49]:
url = 'https://www.nba.com/schedule?cal=October&region=1'

header = {
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.229 Whale/2.10.123.42 Safari/537.36'
}
res = requests.get(url, headers=header)
res

<Response [200]>

In [50]:
soup = BeautifulSoup(res.text, 'html.parser')
soup.head()

[<meta charset="utf-8"/>,
 <link href="/site-manifest.json" rel="manifest"/>,
 <meta content="app-id=484672289" name="apple-itunes-app"/>,
 <meta content="width=device-width,initial-scale=1,minimum-scale=1,maximum-scale=5" name="viewport"/>,
 <script src="/newrelic/newrelic-prod.js"></script>,
 <script defer="" src="https://assets.adobedtm.com/82bf3c13a525/38d894c71c0b/launch-ff3828f362fb.min.js"></script>,
 <link href="https://neulionms-a.akamaihd.net/nbad/player/v1/nba/site_4/styles/nlaaui.css" rel="stylesheet"/>,
 <script src="https://neulionms-a.akamaihd.net/nbad/player/v1/nba/site_4/scripts/nbanlaaeui.js"></script>,
 <script>window.digitalData={"page":{"pageInfo":{"pageName":"nba:schedule:main","siteSection":"nba:schedule","subSection1":"nba:schedule:main","subSection2":"nba:schedule:main","primaryCategory":"schedule"}},"site":{"productName":"nba:web","league":"nba"},"exists":true}</script>,
 <script src="/adfuel/initiator.js"></script>,
 <title>NBA League Schedule for October | N

### 셀레니움으로 쿠키 팝업창 삭제

In [51]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver_path = '/home/rlxdn/Downloads/chromedriver'
driver = webdriver.Chrome(driver_path)
driver.get("https://www.nba.com/schedule?cal=October&region=1")

WebDriverWait(driver, 10).until(
  EC.presence_of_element_located(
      (By.CSS_SELECTOR, "#onetrust-accept-btn-handler")
  )
)

btn = '#onetrust-accept-btn-handler'
driver.find_element_by_css_selector(btn).click()

html = driver.page_source
driver.close()

In [52]:
soup = BeautifulSoup(html)

### 경기 날짜 가져오기

In [53]:
dates = soup.select('h4.text-sm.font-bold.leading-tight.text-cerulean')
date_list = []

for date in dates:
  day = date.get_text()

  date_list.append(day)
date_list

['Sunday, October 3',
 'Monday, October 4',
 'Tuesday, October 5',
 'Wednesday, October 6',
 'Thursday, October 7',
 'Friday, October 8',
 'Saturday, October 9',
 'Sunday, October 10',
 'Monday, October 11',
 'Tuesday, October 12',
 'Wednesday, October 13',
 'Thursday, October 14',
 'Friday, October 15',
 'Tuesday, October 19',
 'Wednesday, October 20',
 'Thursday, October 21',
 'Friday, October 22',
 'Saturday, October 23',
 'Sunday, October 24',
 'Monday, October 25',
 'Tuesday, October 26',
 'Wednesday, October 27',
 'Thursday, October 28',
 'Friday, October 29',
 'Saturday, October 30',
 'Sunday, October 31']

### 경기 장소 가져오기

In [54]:
spots = soup.select("div[class='text-sm']")
place_list = []
loc_list = []

i=1
for spot in spots:
  if i%2!=0:
    place = spot.get_text()
    place_list.append(place)
  else:
    location = spot.get_text()
    loc_list.append(location)
  i = i + 1

print(place_list[:10])
print(loc_list[:10])

['STAPLES Center', 'Scotiabank Arena', 'TD Garden', 'FTX Arena', 'Target Center', 'Paycom Center', 'AT&T Center', 'Moda Center', 'Golden 1 Center', 'STAPLES Center']
['Los Angeles,  CA', 'Toronto,  ON', 'Boston,  MA', 'Miami,  FL', 'Minneapolis,  MN', 'Oklahoma City,  OK', 'San Antonio,  TX', 'Portland,  OR', 'Sacramento,  CA', 'Los Angeles,  CA']


### 경기 팀 가져오기

In [55]:
teams = soup.select("a[class='text-cerulean']")
at_list = []
ht_list = []

i=1
for team in teams:
  if i%2!=0:
    at = team.get_text()
    at_list.append(at)
  else:
    ht = team.get_text()
    ht_list.append(ht)
  i = i + 1

print(at_list[:10])
print(ht_list[:10])

['Brooklyn Nets', 'Philadelphia 76ers', 'Orlando Magic', 'Atlanta Hawks', 'New Orleans Pelicans', 'Charlotte Hornets', 'Utah Jazz', 'Golden State Warriors', 'Phoenix Suns', 'Denver Nuggets']
['Los Angeles Lakers', 'Toronto Raptors', 'Boston Celtics', 'Miami Heat', 'Minnesota Timberwolves', 'Oklahoma City Thunder', 'San Antonio Spurs', 'Portland Trail Blazers', 'Sacramento Kings', 'LA Clippers']


### 경기 시간 가져오기

In [56]:
gtimes = soup.select('span.text-sm.uppercase')
gtime_list = []

for gtime in gtimes:
  time = gtime.get_text()
  gtime_list.append(time)
print(gtime_list[:10])

['3:30 pm ET', '7:00 pm ET', '7:30 pm ET', '7:30 pm ET', '8:00 pm ET', '8:00 pm ET', '8:30 pm ET', '10:00 pm ET', '10:00 pm ET', '10:30 pm ET']


### 데이터 프레임 생성

In [57]:
columns = ['날짜', '장소', '지역', '어웨이팀', '홈팀', '경기시간']
df = pd.DataFrame(list(zip(date_list, place_list, loc_list, at_list, ht_list, gtime_list)), columns=columns)
df

,날짜,장소,지역,어웨이팀,홈팀,경기시간
0,"Sunday, October 3",STAPLES Center,"Los Angeles, CA",Brooklyn Nets,Los Angeles Lakers,3:30 pm ET
1,"Monday, October 4",Scotiabank Arena,"Toronto, ON",Philadelphia 76ers,Toronto Raptors,7:00 pm ET
2,"Tuesday, October 5",TD Garden,"Boston, MA",Orlando Magic,Boston Celtics,7:30 pm ET
3,"Wednesday, October 6",FTX Arena,"Miami, FL",Atlanta Hawks,Miami Heat,7:30 pm ET
4,"Thursday, October 7",Target Center,"Minneapolis, MN",New Orleans Pelicans,Minnesota Timberwolves,8:00 pm ET
5,"Friday, October 8",Paycom Center,"Oklahoma City, OK",Charlotte Hornets,Oklahoma City Thunder,8:00 pm ET
6,"Saturday, October 9",AT&T Center,"San Antonio, TX",Utah Jazz,San Antonio Spurs,8:30 pm ET
7,"Sunday, October 10",Moda Center,"Portland, OR",Golden State Warriors,Portland Trail Blazers,10:00 pm ET
8,"Monday, October 11",Golden 1 Center,"Sacramento, CA",Phoenix Suns,Sacramento Kings,10:00 pm ET
9,"Tuesday, October 12",STAPLES Center,"Los Angeles, CA",Denver Nuggets,LA Clippers,10:30 pm ET


In [58]:
print(f"현지시간 기준 {df['날짜'][0]}일 {df['지역'][0]}의 {df['장소'][0]}에서 {df['어웨이팀'][0]}과 {df['홈팀'][0]}의 경기가 {df['경기시간'][0]}에 예정되어 있습니다.")

현지시간 기준 Sunday, October 3일 Los Angeles,  CA의 STAPLES Center에서 Brooklyn Nets과 Los Angeles Lakers의 경기가 3:30 pm ET에 예정되어 있습니다.
